## 1. 초기 설정

In [2]:
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

torch.manual_seed(777)

training_epochs = 15
batch_size = 100

## 2. training set과 Variable 만들기

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/', 
                          train=True, 
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                          batch_size=batch_size,
                                          shuffle=True)

## 3. model 만들기

In [6]:
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3)

## 4. cost function과 optimizer 만들기

In [7]:
cost_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## 5. model 훈련시키기

In [8]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(mnist_train)
    
    for i, (batch_xs, batch_ys) in enumerate(data_loader):
        X = Variable(batch_xs.view(-1, 28 * 28))
        Y = Variable(batch_ys)
        
        optimizer.zero_grad()
        
        Y_hat = model(X)
        cost = cost_func(Y_hat, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost.data[0]))

print('Learning Finished!')

[Epoch:    1] cost = 0.00297811418
[Epoch:    2] cost = 0.00112320809
[Epoch:    3] cost = 0.000736418937
[Epoch:    4] cost = 0.000527944299
[Epoch:    5] cost = 0.000396567717
[Epoch:    6] cost = 0.00030860008
[Epoch:    7] cost = 0.000260536035
[Epoch:    8] cost = 0.000199463466
[Epoch:    9] cost = 0.000178243194
[Epoch:   10] cost = 0.000138991207
[Epoch:   11] cost = 0.000162190947
[Epoch:   12] cost = 0.000119293531
[Epoch:   13] cost = 0.000120216049
[Epoch:   14] cost = 9.66987063e-05
[Epoch:   15] cost = 9.65625659e-05
Learning Finished!


## 6. model 확인하기

In [9]:
# Test model and check accuracy
X_test = Variable(mnist_test.test_data.view(-1, 28 * 28).float())
Y_test = Variable(mnist_test.test_labels)

prediction = model(X_test)
correct_prediction = (torch.max(prediction.data, 1)[1] == Y_test.data)
accuracy = correct_prediction.float().mean()
print('Accuracy:', accuracy)

# Get one and predict
r = random.randint(0, len(mnist_test) - 1)
X_single_data = Variable(mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float())
Y_single_data = Variable(mnist_test.test_labels[r:r + 1])

print("Label: ", Y_single_data.data)
single_prediction = model(X_single_data)
print("Prediction: ", torch.max(single_prediction.data, 1)[1])

Accuracy: 0.9799
Label:  
 8
[torch.LongTensor of size 1]

Prediction:  
 8
[torch.LongTensor of size 1]

